In [ ]:
# 2023-05-23 - This code was created to quickly backup all of the good `loadedSessPickle.pkl` files from the session folders into the './output' subfolder in the session:
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_12-3-25/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_21-17-16/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-04_21-20-3/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_21-26-8/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_22-4-5/output/loadedSessPickle.pkl already in output folder.
# ...

In [17]:
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 3
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from copy import deepcopy
from datetime import datetime, timedelta
from typing import Optional, Dict, List


## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, FileList
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata, get_files_metadata

# import glob # for finding .whl file after building binary repo
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun

global_data_root_parent_path = find_first_extant_path([Path(r'/media/MAX/Data'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'global_computation_results.pkl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from pyphocorehelpers.gui.Jupyter.simple_widgets import build_global_data_root_parent_path_selection_widget
    
all_paths = [Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')]
global_data_root_parent_path = None
def on_user_update_path_selection(new_path: Path):
	global global_data_root_parent_path
	new_global_data_root_parent_path = new_path.resolve()
	global_data_root_parent_path = new_global_data_root_parent_path
	print(f'global_data_root_parent_path changed to {global_data_root_parent_path}')
	assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
			
global_data_root_parent_path_widget = build_global_data_root_parent_path_selection_widget(all_paths, on_user_update_path_selection)
global_data_root_parent_path_widget

global_data_root_parent_path changed to W:\Data


ToggleButtons(description='Data Root:', layout=Layout(width='auto'), options=(WindowsPath('W:/Data'),), style=…

In [4]:
global_data_root_parent_path

WindowsPath('W:/Data')

In [5]:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

# BATCH_DATE_TO_USE = '2024-06-10_GL' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2024-06-10_Lab' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2024-06-10_Apogee' # used for filenames throught the notebook
BATCH_DATE_TO_USE = '2025-01-07_Apogee' # used for filenames throught the notebook


## Old (Pre 2023-09-21):

In [ ]:
# Use glob to find the first generated .whl file in the dist/ directory
found_whl_files = glob.glob('*.whl')
found_whl_files = [a for a in found_whl_files if not a.endswith('current.whl')] # exclude the symlink from the search

In [ ]:
external_computed_ripple_filepath = session.basepath.joinpath('ripple_df.pkl')
## try the '.ripple.npy' ripples:
active_file_suffix = '.ripple.npy'
external_computed_ripple_filepath = fp.with_suffix(active_file_suffix)



In [ ]:
## Moves all previously saved loadedSessPickle.pkl files to the output folder.
correctly_placed_output_files = []
search_path: Path = Path("/home/halechr/turbo/Data/KDIBA/").resolve()
assert search_path.exists()
for p in search_path.rglob(completed_pipeline_filename):
    if p.parent.name != 'output':
        # If not already in the 'output/' subfolder, move it there.
        print(f'p: {p}')
        curr_output_dir = p.parent.joinpath('output')
        curr_output_dir.mkdir(exist_ok=True)
        # print(f'curr_output_dir: {curr_output_dir}')
        new_path = p.replace(curr_output_dir.joinpath(p.name))
        print(f'\t new_path: {new_path}')
        correctly_placed_output_files.append(p)
    else:
        print(f'p: {p} already in output folder.')
        correctly_placed_output_files.append(p)
    
correctly_placed_output_files

In [ ]:
completed_pipeline_filename

In [8]:

## Build Pickle Path:
pkl_path = 'global_batch_result_2023-06-08.pkl'
csv_path = 'global_batch_result_2023-06-08.csv'
h5_path = 'global_batch_result_2023-06-08.h5'

global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(h5_path).resolve() # Use Default

batch_progress_df = BatchRun.load_batch_progress_df_from_h5(global_batch_result_file_path)
batch_progress_df

updated_batch_progress_df = BatchRun.rebuild_basedirs(batch_progress_df, global_data_root_parent_path)
updated_batch_progress_df

updated_good_only_batch_progress_df = updated_batch_progress_df[updated_batch_progress_df['locally_is_ready']].copy()
updated_good_only_batch_progress_df

AttributeError: Can't get attribute 'SessionBatchProgress' on <module 'pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing' from 'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\pyPhoPlaceCellAnalysis\\src\\pyphoplacecellanalysis\\General\\Batch\\NonInteractiveProcessing.py'>

In [ ]:
# Generate code that can be pasted into the current "ReviewOfWork-*.ipynb" notebook to load the good sessions:
print(",\n".join([a_ctxt.get_initialization_code_string() for a_ctxt in updated_good_only_batch_progress_df['context'].to_list()]))


# 2023-09-21 - Mirror Slow Data files to much faster SSD: , Path(r'/home/halechr/FastData')

In [5]:
import os
from pathlib import Path
import shutil # used in `restore_symlink_folder`
from shutil import copytree # used in `make_specific_items_local`
from typing import Optional, Dict, List
from datetime import datetime, timedelta
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file
from attrs import define, field, Factory


In [ ]:

a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_folder_to_local_folder_containing_symlinks(a_symlink_path, dryrun=False)

In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_folder_to_local_folder_containing_symlinks(a_symlink_path, dryrun=False)

In [ ]:
# make_specific_items_local

# existing_symlink_folder_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one')
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
desired_local_folder_paths: List[Path] = [Path(v) for v in ['/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl', '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl',
                                                            '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl', '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl',
                                                            '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl', '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl',
                                                            ]]
make_specific_items_local(a_symlink_path, desired_local_folder_paths=desired_local_folder_paths, dryrun=True)


In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/fet11-01_12-58-54') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_item_to_full_local_item(a_symlink_path, dryrun=True)


In [ ]:
# existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/11-09_22-4-5/11-09_22-4-5IN.5.numclu')
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl')
print(f'existing_symlink_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_item_to_full_local_item(a_symlink_path, dryrun=False)

In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl')
print(f'existing_symlink_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_item_to_full_local_item(a_symlink_path, dryrun=False)

In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
restore_symlink_folder(a_symlink_path, dryrun=False)

In [6]:
def attribute_is_extant_path(instance, attribute, value):
	if not isinstance(value, Path):
		raise TypeError(f"value must be a pathlib.Path but type(value): {type(value)}")
	if not value.exists():
		raise ValueError(f"value must exists on the filesystem!")
	


@define(slots=False)
class DataRepository:
	short_name: str = field()
	host: str = field()
	root_path: Path = field()

	# def __attrs_post_init__(self):
	# 	assert root_path.exists()
		
		
	def try_find_files(self):
		# Find the files and build the movedicts:
		source_data_root = self.root_path.resolve()
		found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
		found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)
		return found_session_pickle_files, found_global_computation_results_files

data_repos = {'MAX':DataRepository('MAX', host='LNX00052', root_path=Path(r'/media/MAX/Data')),
'FastData':DataRepository('FastData', host='LNX00052', root_path=Path(r'/home/halechr/FastData')),
'gen_scripts':DataRepository('gen_scripts', host='LNX00052', root_path=Path(r'/home/halechr/FastData/gen_scripts/')),
}


included_session_contexts = [
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
	  IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

data_repos_good_session_concrete_folders_dict = {}

data_repos
# run_kdiba_gor01_one_2006-6-08_14-26-15

{'MAX': DataRepository(short_name='MAX', host='LNX00052', root_path=WindowsPath('/media/MAX/Data')),
 'FastData': DataRepository(short_name='FastData', host='LNX00052', root_path=WindowsPath('/home/halechr/FastData')),
 'gen_scripts': DataRepository(short_name='gen_scripts', host='LNX00052', root_path=WindowsPath('/home/halechr/FastData/gen_scripts'))}

In [7]:
curr_good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(data_repos['MAX'].root_path.resolve(), included_session_contexts)
curr_good_session_concrete_folders

AssertionError: global_data_root_parent_path: C:\media\MAX\Data does not exist! Is the right computer's config commented out above?

In [ ]:
included_session_contexts

In [8]:
data_repos_good_session_concrete_folders_dict['FastData'] = ConcreteSessionFolder.build_concrete_session_folders(data_repos['FastData'].root_path.resolve(), included_session_contexts, allow_create_needed_dirs=True)
data_repos_good_session_concrete_folders_dict['FastData']

TypeError: build_concrete_session_folders() got an unexpected keyword argument 'allow_create_needed_dirs'

In [ ]:
good_session_concrete_folders

In [9]:
# copy_dict = ConcreteSessionFolder.build_backup_copydict(data_repos_good_session_concrete_folders_dict['FastData'], backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['global_pkl'])
copy_dict_FastData = ConcreteSessionFolder.build_backup_copydict(data_repos_good_session_concrete_folders_dict['FastData'], backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['local_pkl', 'global_pkl'])
copy_dict_FastData

KeyError: 'FastData'

In [ ]:
moved_files_dict_FastData_files = copy_movedict(copy_dict_FastData)
moved_files_dict_FastData_files

In [ ]:
found_session_pickle_files, found_global_computation_results_files = data_repos['MAX'].try_find_files()
found_session_pickle_files

In [ ]:

source_data_root = Path(r'/media/MAX/Data')
dest_data_root = Path(r'/home/halechr/FastData')
assert source_data_root.exists(), f"source_data_root: {source_data_root} does not exist! Is the right computer's config commented out above?"
assert dest_data_root.exists(), f"dest_data_root: {dest_data_root} does not exist! Is the right computer's config commented out above?"

oldest_modified_date = (datetime.now() - timedelta(days=5))

In [ ]:
# Find the files and build the movedicts:
found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)
file_movedict_session_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_session_pickle_files, only_files_newer_than=oldest_modified_date)
file_movedict_global_computation_results_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_global_computation_results_files, only_files_newer_than=oldest_modified_date)

### Actually perform copy operations. This will take a while

In [ ]:
moved_files_dict_session_pickle_files = copy_movedict(file_movedict_session_pickle_files)
moved_files_dict_session_pickle_files

# moved_files_dict = copy_files(filelist_source, filelist_dest)

In [ ]:
moved_files_dict_global_computation_results_pickle_files = copy_movedict(file_movedict_global_computation_results_pickle_files)
moved_files_dict_global_computation_results_pickle_files

# Various file formats:
'global_computation_results_2023-12-07_GL.pkl'
'20231012125859-global_computation_results.pkltmp'
"loadedSessPickle_2023-12-07_GL.pkl"
"backup-20231020190542-loadedSessPickle.pkl"
"loadedSessPickle_2023-10-25_BidirectionalOnly"
"20230912203733-loadedSessPickle.pkl"
"backup-20240125143331-loadedSessPickle.pkl.bak"
"20231129083822-loadedSessPickle.pkl"


In [10]:
from pyphocorehelpers.Filesystem.path_helpers import parse_unique_file_name
import re
from datetime import datetime

def detect_date_strings(filenames):
    # Regular expression pattern for date (YYYY-MM-DD)
    date_pattern = r'\d{4}-\d{2}-\d{2}'
    # Regular expression pattern for datetime (YYYYMMDDHHMMSS)
    datetime_pattern = r'\d{14}'

    detected_dates = []

    for filename in filenames:
        date_match = re.search(date_pattern, filename)
        datetime_match = re.search(datetime_pattern, filename)

        if date_match:
            date_str = date_match.group()
            # Parse the date string
            try:
                date_value = datetime.strptime(date_str, '%Y-%m-%d')
            except ValueError as e:
                print(f"Error parsing date: {e}")
                date_value = None
            detected_dates.append((date_str, date_value))
        elif datetime_match:
            datetime_str = datetime_match.group()
            # Parse the datetime string
            try:
                datetime_value = datetime.strptime(datetime_str, '%Y%m%d%H%M%S')
            except ValueError as e:
                print(f"Error parsing datetime: {e}")
                datetime_value = None
            detected_dates.append((datetime_str, datetime_value))
        else:
            detected_dates.append((None, None))  # No date/datetime found

    return detected_dates


test_filenames = ['global_computation_results_2023-12-07_GL.pkl',
'20231012125859-global_computation_results.pkltmp',
"loadedSessPickle_2023-12-07_GL.pkl",
"backup-20231020190542-loadedSessPickle.pkl",
"loadedSessPickle_2023-10-25_BidirectionalOnly",
"20230912203733-loadedSessPickle.pkl",
"backup-20240125143331-loadedSessPickle.pkl.bak",
"20231129083822-loadedSessPickle.pkl"]

# [parse_unique_file_name(a_filename) for a_filename in test_filenames]

detected_dates = detect_date_strings(test_filenames)
for filename, (date_str, date_value) in zip(test_filenames, detected_dates):
    print(f'{filename} --> {date_str}, {date_value}')
    

global_computation_results_2023-12-07_GL.pkl --> 2023-12-07, 2023-12-07 00:00:00
20231012125859-global_computation_results.pkltmp --> 20231012125859, 2023-10-12 12:58:59
loadedSessPickle_2023-12-07_GL.pkl --> 2023-12-07, 2023-12-07 00:00:00
backup-20231020190542-loadedSessPickle.pkl --> 20231020190542, 2023-10-20 19:05:42
loadedSessPickle_2023-10-25_BidirectionalOnly --> 2023-10-25, 2023-10-25 00:00:00
20230912203733-loadedSessPickle.pkl --> 20230912203733, 2023-09-12 20:37:33
backup-20240125143331-loadedSessPickle.pkl.bak --> 20240125143331, 2024-01-25 14:33:31
20231129083822-loadedSessPickle.pkl --> 20231129083822, 2023-11-29 08:38:22


In [9]:
from neuropy.core.session.Formats.SessionSpecifications import SessionFolderSpec, SessionFileSpec


def get_pipeline_output_files(session_name: str):

    return SessionFolderSpec(optional=[SessionFileSpec('loadedSessPickle.pkl', session_name, 'The entire computed pipeline', None),
            SessionFileSpec('{}.mua.npy', session_name, 'Multi-Unit Acitivity Cache', None),
            SessionFileSpec('{}.pbe.npy', session_name, 'Population Burst Events Cache', None),
            SessionFileSpec('{}.interpolated_spike_positions.npy', session_name, 'Position-interpolated spike info computed from the spike times and the positions', None),
            SessionFileSpec('{}.position.npy', session_name, 'Interpolated position info with additional properties', None),
            # SessionFolderSpec(optional=[SessionFileSpec(global_computation_results.pkl', session_name, 'The global (across-epoch) computation functions computed results for the pipeline', None),
                # ]
            # ),
            SessionFileSpec('output/global_computation_results.pkl', session_name, 'The global (across-epoch) computation functions computed results for the pipeline', None),
        ]
    )



In [27]:

# Hardcoded included_session_contexts:
from typing import Tuple

included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

# ...
# (WindowsPath('W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/loadedSessPickle.pkl'),
#     {'path': 'W:\\Data\\KDIBA\\vvp01\\one\\2006-4-10_12-25-50\\loadedSessPickle.pkl',
#      'modification_time': datetime.datetime(2024, 12, 16, 19, 43, 45, 291007),
#      'creation_time': datetime.datetime(2023, 6, 6, 23, 16, 16, 393205),
#      'file_size': 4.68}),
# ...

active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/home/halechr/cloud/turbo/Data'), Path(r'W:\Data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

## Build session paths dictionary:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
# found_files_dict: Dict[IdentifyingContext, Tuple[List[Path], List[Path]]] = {}

found_files_dict: Dict[IdentifyingContext, Tuple[List[Tuple[Path, Dict]], List[Tuple[Path, Dict]]]] = {}


for a_ctxt, a_path in session_basedirs_dict.items():
    found_any_kind_session_pickle_files = discover_data_files(a_path, glob_pattern='*loadedSessPickle*.pkl', recursive=True)
    found_any_global_computation_results_files = discover_data_files(a_path, glob_pattern='output/*global_computation_results*.pkl', recursive=True)
    # metadata_dict = {'found_any_kind_session_pickle_files': get_files_metadata(found_any_kind_session_pickle_files),
    #                     'found_any_global_computation_results_files': get_files_metadata(found_any_kind_session_pickle_files)
    # }
    found_any_kind_session_pickle_files = [(f, get_file_metadata(f)) for f in found_any_kind_session_pickle_files]
    found_any_global_computation_results_files = [(f, get_file_metadata(f)) for f in found_any_global_computation_results_files]

    # found_files_dict[a_ctxt] = (found_any_kind_session_pickle_files, found_any_global_computation_results_files)
    found_files_dict[a_ctxt] = (found_any_kind_session_pickle_files, found_any_global_computation_results_files)

found_files_dict


{Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-08_14-26-15'): ([(WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20240926170122-loadedSessPickle.pkl'),
    {'path': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20240926170122-loadedSessPickle.pkl',
     'modification_time': datetime.datetime(2024, 9, 26, 17, 1, 22, 465613),
     'creation_time': datetime.datetime(2024, 9, 26, 17, 1, 22, 461720),
     'file_size': 0.0}),
   (WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20241202120145-loadedSessPickle.pkl'),
    {'path': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20241202120145-loadedSessPickle.pkl',
     'modification_time': datetime.datetime(2024, 12, 2, 12, 1, 49, 962720),
     'creation_time': datetime.datetime(2024, 12, 2, 12, 1, 45, 321199),
     'file_size': 1.81}),
   (WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/bak/20230912203733-loadedSessPickle.pkl'),
    {'path': 'W:\\Data\\KDIBA\\gor01\\one\\

In [28]:
found_pipeline_output_files_dict = {a_session_folder.context:get_pipeline_output_files(a_session_folder.context.session_name) for a_session_folder in good_session_concrete_folders}
found_pipeline_output_files_dict

{Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-08_14-26-15'): <SessionFolderSpec: {'required_files': [], 'optional_files': [SessionFileSpec(fileSpecString='loadedSessPickle.pkl', suggestedBaseName='2006-6-08_14-26-15', description='The entire computed pipeline', session_load_callback=None), SessionFileSpec(fileSpecString='{}.mua.npy', suggestedBaseName='2006-6-08_14-26-15', description='Multi-Unit Acitivity Cache', session_load_callback=None), SessionFileSpec(fileSpecString='{}.pbe.npy', suggestedBaseName='2006-6-08_14-26-15', description='Population Burst Events Cache', session_load_callback=None), SessionFileSpec(fileSpecString='{}.interpolated_spike_positions.npy', suggestedBaseName='2006-6-08_14-26-15', description='Position-interpolated spike info computed from the spike times and the positions', session_load_callback=None), SessionFileSpec(fileSpecString='{}.position.npy', suggestedBaseName='2006-6-08_14-26-15', description='Interpolated 

In [36]:
def render_single_file(file) -> str:
    """ renders a single file and potentially its size """
    leaf_str = ''
    if not isinstance(file, (tuple, list, dict)):
        leaf_str += f"'{file}'"
    else:
        ## handle multi-info files
        file_tuple = file
        assert len(file_tuple) == 2, f"file_tuple: {file_tuple}, len(file_tuple): {len(file_tuple)}"
        file, file_metadata_dict = file_tuple
        formatted_file_metadata_str: str = f"{file_metadata_dict['file_size']} GB"
        leaf_str += f"'{file}'\t{formatted_file_metadata_str}"
    return leaf_str


def render_tree(found_files_dict):
    tree_str = ""
    for session, (session_files, global_files) in found_files_dict.items():
        tree_str += f"{session}\n"
        if session_files:
            tree_str += "  - Session Pickle Files:\n"
            for file in session_files:
                leaf_str: str = render_single_file(file=file)
                tree_str += f"    * {leaf_str}\n"
        
        if global_files:
            tree_str += "  - Global Computation Results Files:\n"
            for file in global_files:
                leaf_str: str = render_single_file(file=file)
                tree_str += f"    * {leaf_str}\n"
    return tree_str

# Example of usage
tree_representation = render_tree(found_files_dict)
print(tree_representation)

kdiba_gor01_one_2006-6-08_14-26-15
  - Session Pickle Files:
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20240926170122-loadedSessPickle.pkl'	0.0 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20241202120145-loadedSessPickle.pkl'	1.81 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\bak\20230912203733-loadedSessPickle.pkl'	0.0 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\bak\20230912203734-loadedSessPickle.pkl'	0.0 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\bak\20231129081520-loadedSessPickle.pkl'	0.0 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\bak\20231129084409-loadedSessPickle.pkl'	0.0 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\bak\loadedSessPickle_2024-03-28_Apogee.pkl'	12.11 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl'	12.25 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle_Pre2024-07-16.pkl'	4.29 GB
    * 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle_withNewComput

In [14]:
import ipywidgets as widgets

def create_tree_view(found_files_dict):
    sessions_accordion = widgets.Accordion(children=[widgets.VBox() for _ in found_files_dict])
    for i, (session, (session_files, global_files)) in enumerate(found_files_dict.items()):
        session_accordion = sessions_accordion.children[i]
        
        # Convert paths to strings and create session pickle files accordion
        session_files_accordion = widgets.Accordion(children=[
            widgets.VBox([widgets.Label(str(file)) for file in session_files])
        ])
        session_files_accordion.set_title(0, 'Session Pickle Files')

        # Convert paths to strings and create global computation results files accordion
        global_files_accordion = widgets.Accordion(children=[
            widgets.VBox([widgets.Label(str(file)) for file in global_files])
        ])
        global_files_accordion.set_title(0, 'Global Computation Results Files')

        # Set session accordion children
        session_accordion.children = [session_files_accordion, global_files_accordion]
        
        # Set main accordion title
        sessions_accordion.set_title(i, str(session))

    return sessions_accordion

# Example usage
tree_view = create_tree_view(found_files_dict)
tree_view


Accordion(children=(VBox(children=(Accordion(children=(VBox(children=(Label(value='W:\\Data\\KDIBA\\gor01\\one…

In [21]:

search_parent_path = Path(r'W:\\Data\\Kdiba')
# search_parent_path = Path(r'/home/halechr/turbo/Data/KDIBA').resolve() # Greatlakes
# search_parent_path = Path(r'/home/halechr/cloud/turbo/Data/KDIBA').resolve() # Lab

# found_autoversioned_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='*-loadedSessPickle.pkl', recursive=True)
# found_global_computation_results_files = discover_data_files(search_parent_path, glob_pattern='output/*.pkl', recursive=True)


found_any_kind_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='*loadedSessPickle*.pkl', recursive=True)
found_any_global_computation_results_files = discover_data_files(search_parent_path, glob_pattern='output/*global_computation_results*.pkl', recursive=True)

# found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
# found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)

found_files = found_any_global_computation_results_files + found_any_kind_session_pickle_files

found_files_metadata = [get_file_metadata(f) for f in found_files]

# found_files_metadata_dict = {f:get_file_metadata(f) for f in found_files}

# found_files
# found_files_metadata_dict
found_files_metadata

pd.DataFrame.from_records(found_files_metadata)

,path,modification_time,creation_time,file_size
0,W:\Data\Kdiba\gor01\one\2006-6-07_11-26-53\out...,2024-10-10 18:07:25.818638,2024-09-09 11:15:22.962073,2.07
1,W:\Data\Kdiba\gor01\one\2006-6-08_14-26-15\out...,2025-01-02 00:53:39.543961,2024-10-04 17:28:08.892762,4.40
2,W:\Data\Kdiba\gor01\one\2006-6-08_14-26-15\out...,2024-09-04 06:19:15.839090,2024-09-03 23:06:23.075876,4.19
3,W:\Data\Kdiba\gor01\one\2006-6-08_14-26-15\out...,2024-09-04 07:35:08.904581,2024-09-04 07:34:43.062705,4.36
4,W:\Data\Kdiba\gor01\one\2006-6-08_14-26-15\out...,2024-10-01 22:19:01.844865,2024-05-01 16:33:04.687235,4.29
...,...,...,...,...
154,W:\Data\Kdiba\vvp01\two\2006-4-19_16-37-40\loa...,2023-07-21 01:56:00.632488,2023-05-15 16:50:15.849116,2.76
155,W:\Data\Kdiba\vvp01\two\2006-4-21_11-19-2\load...,2023-07-21 01:58:30.931844,2023-05-15 16:50:20.343168,3.27
156,W:\Data\Kdiba\vvp01\two\2006-4-25_13-20-55\loa...,2024-09-25 08:34:17.966244,2023-05-15 16:50:25.282259,2.50
157,W:\Data\Kdiba\vvp01\two\2006-4-26_13-51-50\loa...,2023-07-21 02:04:09.629286,2023-05-15 16:50:29.288381,2.77


# 2023-11-20 - Discovery of folder modification times from matching files

In [ ]:
Write a python function that


In [ ]:

def determine_directory_modification_time(target_

# 2024-06-11 - Listing Pickle Contents without Loading everything

In [6]:
# found_session_pickle_files
_test_pickle_dir = Path(r"W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\loadedSessPickle.pkl").resolve()
_test_global_pickle_dir = Path(r"W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\output\global_computation_results.pkl").resolve()

assert _test_pickle_dir.exists()
assert _test_global_pickle_dir.exists()

In [7]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible, debug_dump_object_member_shapes, print_value_overview_only, document_active_variables

_out = loadData(pkl_path=_test_pickle_dir, debug_print=False) # <pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline.NeuropyPipeline at 0x1f5e627c670>
_out

Computing loaded session pickle file results : "W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/loadedSessPickle.pkl"... build_logger(full_logger_string="2025-01-07_00-01-20.Apogee.kdiba.gor01.one.2006-6-12_15-55-31", file_logging_dir: None):
done.


In [8]:

print_keys_if_possible(curr_key='pipeline', curr_value=_out, max_depth=2)

    pipeline: pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline.NeuropyPipeline  = <pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline.NeuropyPipeline object at 0x0000023118684130>
	│   ├── pipeline_name: str  = kdiba_pipeline
	│   ├── session_data_type: str  = kdiba
	│   ├── _stage: pyphoplacecellanalysis.General.Pipeline.Stages.Display.DisplayPipelineStage  = <pyphoplacecellanalysis.General.Pipeline.Stages.Display.DisplayPipelineStage object at 0x000002311865AB50>
		│   ├── stage_name: str  = kdiba_pipeline_input
		│   ├── basedir: pathlib.WindowsPath  = W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31
		│   ├── load_function: NoneType  = None
		│   ├── post_load_functions: list  = [] - (0,)
		│   ├── loaded_data: dict  = {'sess': DataSession(2006-6-12_15-55-31.xml)}
		│   ├── filtered_sessions: dict  = {'maze1_odd': DataSession(2006-6-12_15-55-31.xml), 'maze2_odd': DataSession(2006-6-12_15-55-31.xml), 'maze_odd': DataSession(2006-6-12_15-55-31.xml), 'maze1_even': DataSession(20